<a href="https://colab.research.google.com/github/wangbxj1234/eeg/blob/main/hhtconformer_toydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
import torch.nn as nn

input: (N,C_in,Hin,Win)
output:(N,C_out,Hout,Wout)

Hout=[Hin+2×padding[0]−dilation[0]×(kernel_size[0]−1)−1]/stride[0]

Wout=[Win+2×padding[1]−dilation[1]×(kernel_size[1]−1)−1]/stride[1]

​
 


nn.Conv2d(C_in, C_out, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

In [46]:
#####vanilla shallownet ch=22
shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), (1, 1)),
            nn.Conv2d(40, 40, (22, 1), (1, 1)),#torch.Size([144, 40, 1, 976])
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
            #torch.Size([144, 40, 1, 61])
            nn.Dropout(0.5),
        )

input = torch.randn(144,1, 22, 1000)
output = shallownet(input)
output.size()

torch.Size([144, 40, 1, 61])

1.Patchembedding

In [47]:
######with hht， ch=88
shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), (1, 1)),
            nn.Conv2d(40, 40, (88, 1), (1, 1)),#torch.Size([144, 40, 1, 976])
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
            #torch.Size([144, 40, 1, 61])
            nn.Dropout(0.5),
        )

input = torch.randn(144,1, 88, 1000)    ###toy data
output = shallownet(input)
output.size()

torch.Size([144, 40, 1, 61])

In [48]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
#import einops
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
projection = nn.Sequential(
            nn.Conv2d(40, 40, (1, 1), stride=(1, 1)),  # transpose, conv could enhance fiting ability slightly
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
x1 = projection(output)
x1.size()#torch.Size([144, 61, 40])
###finish the patchembedding

torch.Size([144, 61, 40])

2.TransformerEncoder

In [50]:
from torch import Tensor
import torch.nn.functional as F

In [51]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, num_heads, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.keys = nn.Linear(emb_size, emb_size)
        self.queries = nn.Linear(emb_size, emb_size)
        self.values = nn.Linear(emb_size, emb_size)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        queries = rearrange(self.queries(x), "b n (h d) -> b h n d", h=self.num_heads)
        keys = rearrange(self.keys(x), "b n (h d) -> b h n d", h=self.num_heads)
        values = rearrange(self.values(x), "b n (h d) -> b h n d", h=self.num_heads)
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)  
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1 / 2)
        att = F.softmax(energy / scaling, dim=-1)
        att = self.att_drop(att)
        #out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = torch.einsum('bhqk, bhkv -> bhkv ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out


class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x


class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion, drop_p):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )


class GELU(nn.Module):
    def forward(self, input: Tensor) -> Tensor:
        return input*0.5*(1.0+torch.erf(input/math.sqrt(2.0)))


class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size,
                 num_heads=10,
                 drop_p=0.5,
                 forward_expansion=4,
                 forward_drop_p=0.5):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, num_heads, drop_p),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))


class TransformerEncoder(nn.Sequential):
    def __init__(self, depth, emb_size):
        super().__init__(*[TransformerEncoderBlock(emb_size) for _ in range(depth)])

In [57]:
Encoder = TransformerEncoder(depth=1, emb_size=40)
x2 =Encoder(x1)###TransformerEncoder finished
x2.size()###torch.Size([144, 61, 40])

torch.Size([144, 61, 40])

3.ClassificationHead

In [53]:
class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size, n_classes):
        super().__init__()
        
        # global average pooling
       # self.clshead = nn.Sequential(
       #     Reduce('b n e -> b e', reduction='mean'),
       #     nn.LayerNorm(emb_size),
       #     nn.Linear(emb_size, n_classes)
       # )
        self.fc = nn.Sequential(
            nn.Linear(2440, 256),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 32),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(32, 4)
        )

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        out = self.fc(x)
        return x, out  

In [58]:
Head = ClassificationHead(emb_size=40,n_classes=6)
x3,out =Head(x2)###ClassificationHead finished
x3.size()###torch.Size([144, 2440])
out.size()###torch.Size([144, 4])

torch.Size([144, 4])

In [59]:
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50, 100)
output = m(input)
output.size()

torch.Size([20, 33, 24, 49])

In [60]:
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
input = torch.randn(20, 16, 50, 100)
output = m(input)
output.size()

torch.Size([20, 33, 28, 100])

In [61]:
# non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 100)
output = m(input)
output.size()

torch.Size([20, 33, 26, 100])